<h1 style="padding-top: 25px;padding-bottom: 25px;text-align: left; padding-left: 10px; background-color: #DDDDDD; 
    color: black;"> <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS109A Introduction to Data Science </h1>

## Homework 1: Data Collection, Parsing, and Quick Analyses

**Harvard University**<br/>
**Fall 2021**<br/>
**Instructors**: Pavlos Protopapas and Natesh Pillai<br/>
<hr style='height:2px'>

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2021-CS109A/master/themes/static/css/cs109.css").text
HTML(styles)


## Overview 

In this homework, your goal is to learn how to acquire, parse, clean, and analyze data. Toward this goal, we will address certain questions about COVID, and you will scrape data directly from a website. For the remainder of the semester, we will provide you data files directly; however, since real-world problems often require gathering information from a variety of sources, including the Internet, web scraping is a highly useful skill to have.

### Instructions
- To submit your assignment, follow the instructions given in Canvas.

### Learning Objectives
- Get started using [Jupyter Notebooks](https://jupyter.org/), which are incredibly popular, powerful, and will be our medium of programming for the duration of CS109A and CS109B.
- Become familiar with how to access and use data from various sources (i.e., web scraping and directly from files).
- Gain experience with data exploration and simple analysis.
- Become comfortable with [pandas](https://pandas.pydata.org/) as a means of storing and working with data.
- Reflect on what further analysis you may wish to do with this data. For example, given the material we've covered so far, what *more* do you wish you had the ability to do (e.g., modelling, prediction, etc). That is, think about questions you may have about the data, and try to imagine what types of tools you might need to help answer your questions.

### Notes
- Exercise **responsible scraping**. Web servers can become slow or unresponsive if they receive too many requests from the same source in a short amount of time. In your code, use a delay of 2 seconds between requests. This helps to not get blocked by the target website -- imagine how frustrating it would be to have this occur. Section 1 of this homework involves saving the scraped web pages to your local machine. Thus, after completing Section 1, you do not need to re-scrape any of the pages, unless you wish to occasionally grab the latest data. 

- <span style='color:purple'>**Web scraping requests can take several minutes**</span>. This is another reason why you should not wait until the last minute to do this homework.
- As you run a Jupyter Notebook, it maintains a running state of memory. Thus, <span style='color:purple'>the order in which you run cells matters</span> and plays a crucial role; it can be easy to make mistakes based on *when* you run different cells as you develop and test your code. Before submitting every Jupyter Notebook homework assignment, be sure to restart your Jupyter Notebook and run the entire notebook from scratch, all at once (i.e., "Kernel -> Restart & Run All"). Just make sure to not re-run the time intensive tasks unnecessarily. In this notebook for example, you could declare a variable to act as a 'setting' and use some controll logic to prevent a re-scrap from happening when not desired.

- We will be working with COVID data. COVID has impacted everyone in the world, and naturally some people have been greatly more affected than others. We, the teaching staff, are sensitive to this, empathize, and understand that working with COVID data may be unsettling to some. We apologize for any discomfort this may cause. Our intent with this assignment is purely pedagogical, and we'd like to remind students that data science and machine learning can be used to provide insights that can be used for good and invoke change. Toward this goal, parts of the homework are intended to shed light on the unfortunate, widespread inequality that exists. So, while this data may be unsettling, our aim is for the learned skills addressed here -- and in all future assignments -- to provide you with knowledge and confidence to do good work.

## 1. Obtaining Data (17 points)

For any given situation or scenario that we wish to understand, we will rely on having relevant data. Here, we are interested in the degree to which the SARS-CoV-2 virus has affected United States citizens (SARS-CoV-2 is the virus that causes the COVID-19 disease). The Centers for Disease Control and Prevention (CDC) provides relevant data from USAFacts.org that includes the number of confirmed COVID-19 cases on a per-county basis. Visit https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/. At the bottom of the web page, in a blue table, you should see a list of every state, each of which has its own web page.

In this exercise, we will focus on automating the downloading of each state's data with [Requests](https://docs.python-requests.org/en/master/) and then manipulating it with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). 

But first, as we will do for every Jupyter Notebook, let's import necessary packages that we will use throughout the notebook (i.e., run the cell below). 

In [2]:
# import the necessary libraries
import re
import requests
import pandas as pd
import numpy as np
from time import sleep
from bs4 import BeautifulSoup
import pickle # for loading a dictionary from disk
from typing import Optional # typehint that value can also be None

# NOTE: files will be saved to this directory, so you need to ensure
# that it exists on your system first (it should be visible from the
# directory of where you are running this Notebook file)
# i.e.,
# >> ls
# cs109a_hw1_student.ipynb
# data/
# state_data/
state_dir = "state_data/"

In [3]:
# we define this for convenience, as every state's url begins with this prefix
base_url = 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/'

<div class='exercise'><b> Exercise 1.1 [1 pt]: Fetching Website data via Requests</b>

Fetch the web page located at `base_url` and save the request's returned object (a Response object) to a variable named `home_page`.
</div>

In [4]:
# YOUR CODE HERE
home_page = requests.get(base_url) #using .get from requests to return the response object

# END OF YOUR CODE HERE

<div class='exercise'><b>Exercise 1.2 [2 pts]:</b> In the cell below:
    
- Write a line of code that prints to the screen the status of `home_page` (the web page's returned object). You should receive a code of 200 if the request was successful; then,

- **When working with Jupyter Notebooks, avoiding unnecessarily long output in is essential.** Write code that prints the first 10,000 characters from the contents of `home_page` and [enable scolling output for the cell](https://www.youtube.com/watch?v=U4usAUZCv_c&t=1s).</div>


In [5]:
# YOUR CODE HERE
print(home_page)
home_page.status_code
print(home_page.text[:10000])  # print the first 10000 characters of the response object text using .text


# END OF YOUR CODE HERE

<Response [200]>
<!doctype html><html lang="en"><head><script type="text/javascript">window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"1475026924","applicationTime":1096.242526,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"fe3fd8844fc31fb6","agentToken":null}; (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam-cell.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"1475026924"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n){return function(){return o(t,[u.now

<div class='exercise'><b> Exercise 1.3 [1 pt]:</b>
    
In the cell below, create a new BeautifulSoup object that parses the `home_page` as an HTML document (can be done with 1 line of code)</div>

In [6]:
# YOUR CODE HERE
html_doc = BeautifulSoup(home_page.text,"html.parser") #using beautiful soup to parse the text of home_page using an html parser
print(html_doc.prettify()) #printing to confirm that we have text and that it is pretty

# END OF YOUR CODE HERE

<!DOCTYPE html>
<html lang="en">
 <head>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info = {"agent":"","beacon":"bam-cell.nr-data.net","errorBeacon":"bam-cell.nr-data.net","licenseKey":"NRJS-c11b817f31177e0b4d1","applicationID":"1475026924","applicationTime":1096.242526,"transactionName":"ZwZaNUEFVhZZAkNRWl5Mdg5BCVkJURtSXGBCChdL","queueTime":0,"ttGuid":"fe3fd8844fc31fb6","agentToken":null}; (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam-cell.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-c11b817f31177e0b4d1",applicationID:"1475026924"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n){return function(){return o(t,[u.now()].con

<div class='exercise'><b> Exercise 1.4 [8 pts]:</b>
    
In the cell below, write code that uses the BeautifulSoup object to parse through the home page in order to extract the link for every state. Feel free to use [Regular Expressions]('https://docs.python.org/3/library/re.html'), in conjunction with any BeautifulSoup parsing. Specifically, the goal is to populate a `state_urls` [dictionary]('https://docs.python.org/3/tutorial/datastructures.html#dictionaries') by setting each key to be the state name and the value to be the full URL. When complete, there will be 51 keys (50 states + 1 for DC).

### AS A CRITICAL EXAMPLE:
Within `state_urls`, one of your <key, value> pairs should be:

``"District of Columbia" : "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia"``

The casing here is **incredibly** important because later, in Exercise 4, you will merge your data with another dataset that has casing of this form. Thus, our key here should be `District of Columbia` and not `District Of Columbia` or `district-of-columbia`.


**NOTES:**
- There are _many_ solutions, but you may find it easiest to use Regular Expression(s)
- Pay attention to the casing example above, so that your later exercises go smoothly.
- Some HTML tag attributes may change over time. It your code stops working, make sure you are not targeting such ephemeral elements ('jss' class attributes are a common culprit)
</div>

In [7]:
state_urls = {} #initializing state url dictionary 
#Base url would duplicate /visualizations/coronavirus-covid-19-spread-map 

# YOUR CODE HERE
states = html_doc.find_all('a', class_= "MuiTypography-root MuiLink-root MuiLink-underlineAlways MuiTypography-colorInherit")
for i in states:  #create for loop to cycle between each line in file to get the state, url
    state_name = i.get_text()  #finding state name in each line 
    state_urls[state_name] = str("https://usafacts.org") + i["href"] #adding in the base url as a string and adding it to the url by looking for href
    
state_urls #display results to confirm its correct
    


# END OF YOUR CODE HERE

{'Alabama': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/alabama',
 'Alaska': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/alaska',
 'Arizona': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/arizona',
 'Arkansas': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/arkansas',
 'California': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/california',
 'Colorado': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/colorado',
 'Connecticut': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/connecticut',
 'Delaware': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/delaware',
 'District of Columbia': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia',
 'Florida': 'https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/st

Run the cell below to help ensure your formatting is correct and has 51 <key, value> pairs.

In [8]:
# SANITY CHECK
if len(state_urls.keys()) != 51 or \
state_urls["District of Columbia"] != "https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/district-of-columbia":
    print("** 1.4 is incorrect")
else:
    print("** 1.4 might be correct")

** 1.4 might be correct


We wish to use the data without having to re-download it every time. So, let's save each webpage to our local hard drive. **NOTE: It's probably okay to download all of the state web pages a few times a day, but it's safer to keep it to a minimum.**

<div class='exercise'><b> Exercise 1.5 [5 pts]:</b>
    
In the cell below, we will iterate through all <key, value> items in `state_urls`. Your job is to make a web request for each URL and save the **contents** out to a file on your hard drive (use `state_dir`, defined above, as the prefix to the path.) 

**NOTES:**
- **Leave a 2 second pause between requests**
- You should be saving to a file the actual content of the webpage, not a BeautifulSoup object. That is, you should be able to open the saved files in an editor and see the HTML code, just as you could if you were to view the webpage in your browser and click 'View Page Source'.
- See [official Python documentation](https://docs.python.org/3/tutorial/inputoutput.html#reading-and-writing-files) for details on how to read/write files to disk
- You should have saved 51 different files to your hard drive.
- **Once you have written the files you can comment out this cell. This will save time and prevent you from making unnecessary requests when you restart the kernel & re-run all cells in the noteboook before submitting (as you should!)**
</div>

In [9]:
# 1.5 (5 pts) -- save each webpage to disk
# for state, url in state_urls.items(): #for loop to go through each dict key, value
    
    # YOUR CODE HERE
    
    # with open(state_dir + state, 'w') as statefile: # creating new write files for each state and placing them in state_data/
        # statefile.write(requests.get(url).text) # writing out the response object text within each associated state file

    # END OF YOUR CODE HERE
    
    # sleep(2) # LEAVE THIS IN

## 2. Loading and Exploring Data (22 pts)
Now, let's actually use the data! Fortunately, it's saved to our local machine, so we don't need to re-crawl the data every time we wish to access it. We want you to understand that [pandas](https://pandas.pydata.org/) is a library of useful data structures and operations, but we also wish to remind you that it isn't magic and it isn't the _only_ way to do Data Science; it's just a tool to help, and you could do the same operations without pandas. Thus, here we ask you to perform a few operations without using pandas, and then in Exercise 3 we will use pandas.

**Terminology Notice:** In the United States, every state is comprised of many **counties.** You can think of a **county** as being a pretty large district. 

First, run the cell below to construct `state_info`
This is an example of a Python [list comprehension](https://docs.python.org/3/tutorial/datastructures.html#list-comprehensions).

In [10]:
state_info = [(state, state_dir + state) for state in state_urls.keys()]

<div class='exercise'><b> Exercise 2.1 [10 pts]: Parsing and storing data</b>
    
Complete the `load_covid_data()` function, which:

- Takes as input `state_info`, which is a list of [tuples](https://docs.python.org/3.3/library/stdtypes.html?highlight=tuple#tuple): (state name, path to the corresponding file)
- Parses the contents of the file and extracts for **each county**:
    - 7 day average case
    - 7 day average deaths
    - \# of confirmed cases (total)
    - \# of deaths
    - Stores the above 4 pieces of data above as well as **population** in a **non-pandas** data structure named `covid_data` **for every county across every state**
- Returns `covid_data`
    <font color='blue'>


**NOTES:**
- **Attention: the population variable not in `state_info`. More on info on where to get this value is found in the green block below**
- To be clear, as of September 7, 2021, the webpage for Alabama currently lists 67 counties. District of Columbia has 1 county, and Wyoming has 23. Here we are asking you to store in `covid_data` *all counties* across every state. So, later, if we were wished to access just Wyoming's information, you could easily retrieve such for each of its 23 counties, or the info for any of the 67 counties in Alabama.
- `covid_data` **must not be a PANDAS data structure;** it must use a combination of lists and/or dictionaries. It's up to you to decide how to organize this, e.g., a lists of lists of lists, or a list of dictionaries, or a dictionary of dictionaries, or a dictionary of lists of lists, etc. A guiding decision should be ease of access for computing basic stats (Exercises 2.2, 2.3, and 2.4)
- For the duration of our using this data for the homework, be sure to **properly store the data with the correct data types;** that is, counts should be represented as Integers and rates should be represented as Floats. For example:
    - \# of confirmed cases (total) should be an **Integer**
    - \# of deaths should be an **Integer**
    - \# of confirmed cases (per 100k) should be a **Float** (we haven't created this feature yet!)
    - 7 day average cases should be an **Integer** (you'd think an average should be a float but the values you scrapped were rounded to the nearest int)
</div>

<div style='background-color:lightgreen;padding:15px'>
    <strong>Injecting population data</strong>
    

The table on usafacts.org you've just scrapped originally had additional columns related to county population. But these have recently been removed! We'd like you to be able to utilize the population data in the following section but also use up-to-date COVID data (so the [Internet Archive](https://archive.org/) was not an option). And, though this information is available elsewhere on usafacts.org, we've decided that you've already done enough web scraping for one HW. So below we've provided a [kludge](https://en.wikipedia.org/wiki/Kludge#Computer_science).
    
`population_dict` is a nested dictionary. The keys are states whose values are _themselves_ dictionaries. Those '_inner_' dictionaries' keys are counties and their values are populations. It looks like this:
```python
{'Alabama': {'Autauga County': 55869,
             'Baldwin County': 223234,
           ...
'Wyoming': {'Albany County': 38880,
            'Big Horn County': 11790,
            ...
```

To get at a population you could use double dictionary indexing like `population_dict['Alabama']['Autauga County']`

But not all of the counties you've scrapped have population data in this dictionary. So we've provided a helper function, `get_pop`, that will return `None` if the county data was not found. Use `get_pop` to inject popoulation data into your `covid_data` as you build it up in the `load_covid_data` function you'll implement below.
    
**Final Note: you should _ignore counties with missing population data or populations of 0_. Simply do not add them to `covid_data` as it is constructed.**
</div>

In [11]:
# load additional county population data as a nested dictionary
# you can read about this strange .pkl 'pickle' file here
# https://docs.python.org/3/library/pickle.html
with open('population.pkl', 'rb') as f:
    population_dict = pickle.load(f)

# not sure what's happening with the data types in the function header?
# check out: https://docs.python.org/3/library/typing.html#module-typing
def get_pop(state: str, county: str) -> Optional[int]:
    '''
    returns population of country, state (int)
    If county or state not found, returns None
    Example: get_pop('Alabama', 'Autauga County')
    '''
    try:
        return population_dict.get(state).get(county)
    except AttributeError:
        print('incorrect state name!')
        return None

In [12]:


def load_covid_data(state_info):
    covid_data = {}
    

    # YOUR CODE HERE
    for state, path in state_info:
        
        state_name = open (path, 'r') # now reading out the path data 
        read_data = state_name.read()   # reading out the data within each files
        parse_data = BeautifulSoup(read_data,"html.parser") #Parse the html using beautiful soup
        table_body = parse_data.find_all('tbody')[1]  #looking for the second table using tbody
        rows = table_body.find_all('tr')  # Pulling out specific rows within the table


    # print(rows[1])
        county_dict = {} # creating county dictionary 
    
        for i in rows:                           # Need a for loop to iterate through each county name & data values
    
            county_name = i.find('a').get_text()     # Getting county names and turning it into text
            values = i.find_all('td')                # Getting all the county data
            if get_pop(state, county_name) != 0 and get_pop(state, county_name) != None:
                
            
                
                
            
                data_dict = {}                           # Dictionary to store the county's data
    
                data_dict["seven_day_case_avg"] = int(values[0].get_text().replace(',',''))   
                data_dict["seven_day_death_avg"] = int(values[1].get_text().replace(',',''))
                data_dict["total_cases_per_100k"] = int(values[2].get_text().replace(',',''))/get_pop(state, county_name)*100000
                data_dict["total_cases"] = int(values[2].get_text().replace(',',''))
                data_dict["total_deaths"] = int(values[3].get_text().replace(',',''))
                data_dict["population"] = get_pop(state, county_name)
            
                county_dict[county_name] = data_dict        # appending the county data to our Key of the county names
            covid_data[state] = county_dict    


    # for state, directory in state_info: 
        

    # END OF YOUR CODE HERE
    return covid_data

load_covid_data(state_info)


{'Alabama': {'Autauga County': {'seven_day_case_avg': 43,
   'seven_day_death_avg': 0,
   'total_cases_per_100k': 16479.621972829296,
   'total_cases': 9207,
   'total_deaths': 119,
   'population': 55869},
  'Baldwin County': {'seven_day_case_avg': 134,
   'seven_day_death_avg': 3,
   'total_cases_per_100k': 15555.874105199031,
   'total_cases': 34726,
   'total_deaths': 411,
   'population': 223234},
  'Barbour County': {'seven_day_case_avg': 17,
   'seven_day_death_avg': 0,
   'total_cases_per_100k': 13205.865672851009,
   'total_cases': 3260,
   'total_deaths': 67,
   'population': 24686},
  'Bibb County': {'seven_day_case_avg': 25,
   'seven_day_death_avg': 0,
   'total_cases_per_100k': 16754.487809234615,
   'total_cases': 3752,
   'total_deaths': 74,
   'population': 22394},
  'Blount County': {'seven_day_case_avg': 47,
   'seven_day_death_avg': 0,
   'total_cases_per_100k': 15728.219140179159,
   'total_cases': 9095,
   'total_deaths': 146,
   'population': 57826},
  'Bullock C

Run the cell below (no changes necessary) to execute your code above

In [13]:
covid_data = load_covid_data(state_info)

In [14]:
population_dict

{'Alabama': {'Autauga County': 55869,
  'Baldwin County': 223234,
  'Barbour County': 24686,
  'Bibb County': 22394,
  'Blount County': 57826,
  'Bullock County': 10101,
  'Butler County': 19448,
  'Calhoun County': 113605,
  'Chambers County': 33254,
  'Cherokee County': 26196,
  'Chilton County': 44428,
  'Choctaw County': 12589,
  'Clarke County': 23622,
  'Clay County': 13235,
  'Cleburne County': 14910,
  'Coffee County': 52342,
  'Colbert County': 55241,
  'Conecuh County': 12067,
  'Coosa County': 10663,
  'Covington County': 37049,
  'Crenshaw County': 13772,
  'Cullman County': 83768,
  'Dale County': 49172,
  'Dallas County': 37196,
  'DeKalb County': 71513,
  'Elmore County': 81209,
  'Escambia County': 36633,
  'Etowah County': 102268,
  'Fayette County': 16302,
  'Franklin County': 31362,
  'Geneva County': 26271,
  'Greene County': 8111,
  'Hale County': 14651,
  'Henry County': 17205,
  'Houston County': 105882,
  'Jackson County': 51626,
  'Jefferson County': 658573,
  

<div class='exercise'><b> Exercise 2.2 [4 pts]: Simple analytics</b>
    
Complete the `calculate_county_stats()` function, which calculates:
1. The single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. The single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people
   
**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide
- These values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).
- If there are ties, return any one of the tied counties (see if you can do it in an unbiased way!)
</div>

In [15]:
def calculate_county_stats(covid_data):
    
    # YOUR CODE HERE
    min_rate = float(100000) #setting a high value to compare against the county rates
    max_rate = float(0) #setting a low value to compare against the county rates

    for state in covid_data: 
        for county in covid_data[state]:   
            
            if covid_data[state][county]['total_cases_per_100k'] < min_rate :  #if a county min rate beats current min rate
                min_rate = covid_data[state][county]['total_cases_per_100k'] #change the minimum rate to the new rate
                min_county = county  #store county that has the minimum rate
                min_state = state   #store state that has the minimum rate
   
    

            if covid_data[state][county]['total_cases_per_100k'] > max_rate : #if a county max rate beats current max rate
                max_rate = covid_data[state][county]['total_cases_per_100k']
                max_county = county #store county that has the minimum rate
                max_state = state #store state that has the minimum rate
    
    
    print("This is the county with the lowest total case rate: " + min_county + ", "+ min_state + " rate is " + str(min_rate))
    print("This is the county with the highest total case rate: " + max_county + ", "+ max_state + " rate is " + str(max_rate))
    

    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [16]:
calculate_county_stats(covid_data)

This is the county with the lowest total case rate: Lake and Peninsula Borough, Alaska rate is 0.0
This is the county with the highest total case rate: Bristol Bay Borough, Alaska rate is 72727.27272727274


<div class='exercise'><b> Exercise 2.3 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths()` function, which calculates:
1. The state that has the **lowest number** of deaths
2. The state that has the **highest number** of deaths

**NOTES:**
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- These values you report should be Integers, not Floating point numbers.
- If there are ties, return any of the tied states

</div>

In [17]:
def calculate_state_deaths(covid_data):
    
    # YOUR CODE HERE
   
    min_death = 10**6  # setting a high value up to compare to total deaths in the state 
    max_death = 0  # setting a low value up to compare to total deaths in the state 
    
    
    for state in covid_data:
        county_sum = 0  # initializing county_sum to sum
        for county in covid_data[state]:
            county_sum += covid_data[state][county]['total_deaths'] # adding each counties total death to the overall state
        
        if county_sum < min_death: # if current county sum is less than min death amount
            min_death = county_sum # replace if its lower
            min_state = state  # correlate the state with the sum
        
        if county_sum > max_death: # comparing county sum to max death value
            max_death = county_sum  # replace max_death with current county sum if that beats the current value
            max_state = state  # correlate state to the max_death rate 
            
    print("This is the state with the lowest deaths: " + min_state + " with " + str(min_death)+ " deaths")
    print("This is the state with the highest deaths: " + max_state +" with " + str(max_death) + " deaths")
       
            
            
        
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [18]:
calculate_state_deaths(covid_data)

This is the state with the lowest deaths: Hawaii with 145 deaths
This is the state with the highest deaths: California with 65635 deaths


<div class='exercise'><b> Exercise 2.4 [4 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate()` function, which calculates:
1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- To calculate a state's population, we are asserting that is sufficient to sum the population over all counties, and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- **If a county has reported 0 COVID cases,** then we should ignore this county as we estimate its county population. Thus, that county would contribute 0 to its state population total.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [19]:
def calculate_state_deathrate(covid_data):
    
    # YOUR CODE HERE
    min_deathrate = float(0) # set a minimum death rate value
    max_deathrate = float(1e12) # set a maximum death rate value
    min_state = "state1" # set a minimum state value
    max_state = "state2" # set a maximum state value
    
    
    for state in covid_data:  # loop through all the states
        death_count = 0        # intialize death_count
        pop_count = 0          # intialize pop_count
        deathrate = float(0)   # intialize death rate
        
        for county in covid_data[state]:  # iterate through the counties
            if [covid_data[state][county]['total_deaths'] != 0]: # check if county has 0 covid cases, and don't include if it does
                death_count += covid_data[state][county]['total_deaths']  # add total deaths from each county to the county_sum
                pop_count += covid_data[state][county]['population'] 
        deathrate = float(pop_count)/float(death_count) # creating death rate in people per death
        
        
        if deathrate > min_deathrate :  #if statement to compare current death rate to our saved min death rate (lower deathrate will be higher number of people per death)
            min_deathrate = deathrate
            min_state = state
        
        if deathrate < max_deathrate :  #if statement to compare current death rate to our saved max death rate (lower deathrate will be higher number of people per death)
            max_deathrate = deathrate
            max_state = state
            
    # convert deathrates to int here in case two states only differ by a fraction of a person. 
    min_deathrate = int(min_deathrate)
    max_deathrate = int(max_deathrate)
    
    print("This is the state with the lowest death rate: " + min_state + " with a death rate of 1 per "+ str(min_deathrate))
    print("This is the state with the highest death rate: " + max_state + " with a death rate of 1 per "+ str(max_deathrate))
       
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [20]:
calculate_state_deathrate(covid_data)

This is the state with the lowest death rate: Hawaii with a death rate of 1 per 3043
This is the state with the highest death rate: New Jersey with a death rate of 1 per 334


## 3. PANDAS (36 pts)
What if we wanted to observe more than just the single-most extreme counties and states? What if we wanted to inspect all states, after having sorted the data by some feature? As you saw in the above exercises, doing the most basic analytics is possible, but it can quickly become cumbersome. As we learned in class, PANDAS is a great library that provides data structures that are highly useful for data analysis.

<div class='exercise'><b> Exercise 3.1 [10 pts]: Converting to PANDAS</b>

In Exercise 2, we worked with `covid_data`, which is comprises of some combination of lists and/or dictionaries.

Complete the `convert_to_pandas()` function, which converts `covid_data` to a PANDAS DataFrame, whereby:
- Each row corresponds to a unique county
- The 4 columns are:
    - county
    - state
    - \# total covid cases (Integer)
    - \# case per 100k (Integer)
    - \# covid deaths (Integer)
- The columns should be titled **exactly** as listed above

**NOTE:**
- If there exists multiple counties with the same name, each of which belonging to a different state, then there should be a distinct row for each.
- The 2 columns that correspond to COVID counts should all be Integers (e.g., 1498), not Floating point digits (e.g., 1498.0)
</div>

In [21]:
def convert_to_pandas(covid_data):
    
    # YOUR CODE HERE
    
    new_d = {} #create an empty dictionary
    counter = 0
    
    for state in covid_data: # loop over state
        for county in covid_data[state]: # loop over county
            new_d[counter] = covid_data[state][county] # pulling out data points within county index dictionary
            new_d[counter]['state'] = state # place state names in the new dictionary
            new_d[counter]['county'] = county # place county names in the new dictionary
            counter += 1
            
    covid_df = pd.DataFrame.from_dict(new_d, orient = 'index').drop(columns = ['seven_day_case_avg','seven_day_death_avg','population'])
    #created the panda data frame and dropped two columns
    
        
    # Worked with the TA Shuheng on this
    # END OF YOUR CODE HERE
    return covid_df


Run the cell below (no changes necessary) to execute your code above and inspect the results.

In [22]:
covid_df = convert_to_pandas(covid_data)

In [23]:
covid_df.head()

,total_cases_per_100k,total_cases,total_deaths,state,county
0,16479.621973,9207,119,Alabama,Autauga County
1,15555.874105,34726,411,Alabama,Baldwin County
2,13205.865673,3260,67,Alabama,Barbour County
3,16754.487809,3752,74,Alabama,Bibb County
4,15728.219140,9095,146,Alabama,Blount County


In [24]:
covid_df.shape

(3081, 5)

<div class='exercise'><b> Exercise 3.2 [5 pts]: Simple analytics</b>

Complete the `calculate_county_stats2()` function, **which should obtain identical information (other than ties) as problem 2.2, but now using the PANDAS `covid_df` DataFrame.**

That is, it should calculates:
1. the single county (and the state to which it belongs) that has the **lowest rate** of COVID cases per 100k people
2. the single county (and the state to which it belongs) that has the **highest rate** of COVID cases per 100k people

**NOTES:**
- If there are ties, return any of the tied counties
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Floating point numbers (e.g., 3.4), not Integers (e.g., 3).

</div>

In [25]:
def calculate_county_stats2(covid_df):

    # YOUR CODE HERE
    # Need to convert to get RATE, but dropped covid data
    
    highest_rate = covid_df.iloc[covid_df['total_cases_per_100k'].idxmax()]  # pull out the data from associated row
    highest_county = highest_rate.iloc[4] # pull county
    highest_state = highest_rate.iloc[3] # pull state
    
    
    lowest_rate = covid_df.iloc[covid_df['total_cases_per_100k'].idxmin()]  # pull out the data from associated row
    lowest_county = lowest_rate.iloc[4] # pull county
    lowest_state = lowest_rate.iloc[3] # pull state
    
    
    print("This is the county with the highest total case rate: "+ str(highest_rate.iloc[0])+ " in " + highest_county + ", " + highest_state)
    print("This is the county with the lowest total case rate: " + str(lowest_rate.iloc[0])+ " in " + lowest_county + ", " + lowest_state)

    # END OF YOUR CODE HERE

In [26]:
covid_df.iloc[covid_df['total_cases_per_100k'].idxmax()]

total_cases_per_100k           72727.272727
total_cases                             608
total_deaths                              2
state                                Alaska
county                  Bristol Bay Borough
Name: 70, dtype: object

Run the cell below (no changes necessary) to execute your code above

In [27]:
calculate_county_stats2(covid_df)

This is the county with the highest total case rate: 72727.27272727274 in Bristol Bay Borough, Alaska
This is the county with the lowest total case rate: 0.0 in Lake and Peninsula Borough, Alaska


<div class='exercise'><b> Exercise 3.3 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deaths2()` function, **which should obtain identical information as problem 2.3 (other than ties), but now using the PANDAS `covid_df` DataFrame.**
1. the state that has the **lowest number** of deaths
2. the state that has the **highest number** of deaths

**NOTES:**
- If there are ties, return any of the tied states
- Place your resulting variables within the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
- The values you report should be Integers, not Floating point numbers.
</div>

In [48]:
def calculate_state_deaths2(covid_df):
    
    # YOUR CODE HERE
    
        state_totals = covid_df.groupby(by = ["state"], axis = 0).sum() # find total deaths per state
        
        max_death_state = state_totals['total_deaths'].idxmax() # find name of state with most deaths 
        max_death_value = state_totals['total_deaths'].max() # find number of deaths in state with most deaths 
        
        min_death_state = state_totals['total_deaths'].idxmin() # find name of state with least deaths 
        min_death_value = state_totals['total_deaths'].min() # find number of deaths in state with least deaths 
        
        print("This is the state with the lowest deaths: " + min_death_state + " with " + str(min_death_value) + " deaths")
        print("This is the state with highest deaths: " +  max_death_state + " with "+ str(max_death_value)+ " deaths")
       
    # END OF YOUR CODE HERE

Run the cell below (no changes necessary) to execute your code above

In [29]:
calculate_state_deaths2(covid_df)

This is the state with the lowest deaths: Hawaii with 145 deaths
This is the state with highest deaths: California with 65635 deaths


<div class='exercise'><b> Exercise 3.4 [5 pts]: Simple analytics</b>
    
Complete the `calculate_state_deathrate2()` function, **which should obtain identical information as problem 2.4, but now using the PANDAS `covid_df` DataFrame.** That is, return:

1. The state that has the **lowest rate** of deaths based on its entire population
2. The state that has the **highest rate** of deaths based on its entire population

**NOTES:**
- Just as in, 2.4, to calculate a state's population, we are asserting that is sufficient to sum the population over all counties -- and that each county's population can be calculated simply from the data fields stored within `covid_data`.
- Just as in 2.4, counties with 0 COVID cases should contibute 0 to the total population of the state.
- Round your results to the a single person (e.g., "1 out of every 2703 people has died" not 2703.4)
- Place your resulting variables within the blanks of the `print()` statements that we provide (don't just manually type your textual answers in the blanks)
</div>

In [30]:
def calculate_state_deathrate2(covid_df):        
    
    # YOUR CODE HERE
    
    # First try
    #get_pop = pd.DataFrame.from_dict(population_dict, orient = 'columns').sum()
    #county_sum = covid_df.groupby(by = ["state"], axis = 0).sum()
    #deathrate = get_pop/county_sum['total_deaths']
    
    # Second try
    pop = (covid_df['total_cases']*100000/covid_df['total_cases_per_100k']) # calculate population based off existing variables
    temp = covid_df.copy() # create a copy of covid_df to add population column to
    temp.insert(0, 'pop', pop) # add population column
    
    county_sum = temp.groupby(by = ["state"], axis = 0).sum() # sum the columns per each state

    deathrate = county_sum['pop']/county_sum['total_deaths'] # calculate people per death
    
    max_deathrate = int(deathrate.min()) # lowest people per death is highest death rate
    max_state = deathrate.idxmin() # get state name for highest death rate
    

    min_deathrate = int(deathrate.max()) # highest people per death is lowest death rate
    min_state = deathrate.idxmax() # get state name for lowest death rate
    
    print(min_state + " has the lowest COVID death rate; 1 out of every " + str(min_deathrate) + " people has died")
    print(max_state + " has the highest COVID death rate; 1 out of every " + str(max_deathrate) + " people has died")
    
    # END OF YOUR CODE HERE   

    

Run the cell below (no changes necessary) to execute your code above

In [31]:
calculate_state_deathrate2(covid_df)

Hawaii has the lowest COVID death rate; 1 out of every 3042 people has died
New Jersey has the highest COVID death rate; 1 out of every 334 people has died


These are highly alarming and tragic statistics, and doing calculations like this can really put the severity of the virus into a grounded perspective. In order to perfectly understand the virus and its spread, everyone would be tested and we would have contact tracing. Without getting into socio-political issues, our point is that (1) we wish to better understand the virus' effects; (2) naturally, any real-world data is messy, and thus we will never have _perfect_ data.


Let's now attempt to understand _some_ of the uncertainty around our COVID data. It's reasonable to believe that the # of COVID deaths is fairly reliable. That is, there are inevitably some false negatives -- people who died of COVID but were not accounted for, as other conditions were listed as the cause. However, the number of false positives is probably minimal -- if someone was denoted as dying from COVID, it's probably true. It's also the case that every disease has a mortality rate. For example, if 1,000 randomly-selected people contracted COVID, $N\%$ of them will die. We'd imagine that this percentage should be pretty constant throughout all people in the United States. Of course, we can think of reasons for this rate to not be perfectly consistent, as some people are at higher risk (e.g., older folks, people with pre-existing conditions, etc). Yet, we can imagine that this natural *variance* in the population to be fairly uniform throughout the USA at large. To this end, if all counties were equal in their **testing**, we ought to see a consistent ratio between: (a) the # of people who died from COVID; and (b) the # of people who tested positive for COVID. Within the medical domain, this ratio is referred to as the `case_fatality_rate`. For example, if 750 people tested positive for COVID, and 75 of those people died, then our `case_fatality_rate` would be 0.1 (meaning 10%).

<div class='exercise'><b>Exercise 3.5 [5 pts]: Further analytics</b>
    
Complete the `add_death_stats()` function below, which should add 3 new columns:
- `case_fatality_rate`
- `# covid deaths per 100k` and
- `population`

And return the updated DataFrame **sorted by `case_fatality_rate` in ascending order** 

**NOTES:**

- `add_death_stats()` should return a new DataFrame that has 8 columns:
    - county
    - state
    - population
    - \# total covid cases
    - \# covid cases per 100k
    - \# covid deaths
    - \# covid deaths per 100k
    - case_fatality_rate
- DataFrame should be sorted by `case_fatality_rate` in ascending order
- Again, the values for `case_fatality_rate` should be < 1. A value of 1 would mean that 100% of people who tested positive for COVID also died.
- `# covid deaths per 100k` is simply defined as the # of COVID deaths for every 100,000 people. We calculate this on a per-county basis.
- Make sure you inspect your results thoroughly. You may have to address the results of divisions by zero (or prevent these divisions in the first place). 
</div>

In [32]:
def add_death_stats(covid_df):
    
    # can add an infintesimal or fillna after the fact to handle nans from divide by 0.
    
    # YOUR CODE HERE
    
    population = covid_df['total_cases']*100000/covid_df['total_cases_per_100k'] # calculate population using existing variables 
    covid_df.insert(0,"population",population) # add population to covid_df
    deaths_per_100k = covid_df['total_deaths']/covid_df['population']*100000 # calculate deaths per 100k 
    covid_df.insert(0,"total_deaths_per_100k",deaths_per_100k) # add deaths per 100k to covid_df
    case_fatality_rate = covid_df['total_deaths']/covid_df['total_cases'] # calculate case fatality rate
    covid_df.insert(0,"case_fatality_rate",case_fatality_rate) # add case fatality rate to covid_df
    covid_df = covid_df.sort_values(by = ['case_fatality_rate']) # sort rows by case fatality rate
    covid_df = covid_df.dropna() # drop rows with NaN

    
    covid_df.head()

    # END OF YOUR CODE HERE
    return covid_df

Run the cell below (no changes necessary) to execute your code above

In [33]:
covid_updated = add_death_stats(covid_df)
covid_updated


,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,state,county
1674,0.000000,0.000000,3380.0,6479.289941,219,0,Nebraska,Harlan County
1637,0.000000,0.000000,465.0,4516.129032,21,0,Nebraska,Blaine County
259,0.000000,0.000000,1392.0,5172.413793,72,0,Colorado,Jackson County
1690,0.000000,0.000000,664.0,6325.301205,42,0,Nebraska,Loup County
1311,0.000000,0.000000,5463.0,3752.516932,205,0,Minnesota,Cook County
...,...,...,...,...,...,...,...,...
417,0.080313,548.410580,20605.0,6828.439699,1407,113,Georgia,Dodge County
528,0.083612,579.038796,8635.0,6925.303995,598,50,Georgia,Wilcox County
1670,0.097561,642.054575,623.0,6581.059390,41,4,Nebraska,Grant County
434,0.110465,639.515315,2971.0,5789.296533,172,19,Georgia,Glascock County


<div class='exercise'><b>Reflection:</b> Data Analysis allows us to better understand a system or scenario.
</div>

<div class='exercise'><b>Exercise 3.6.1 [2 pts] Trends</b>
    
Having looked at the results from Exercises 3.3, 3.4, and 3.5, what are some trends you've noticed and any conclusions you have? (2-3 sentences)?</div>

<div style='background-color:#F6FEFA;padding:15px'>

Death total is not necessarily predictive of deaths per 100k or case fatality rate. Each of these show different ways of how to measure impact of covid in a community, but they are each sensitive to bias. 

</div>


<div class='exercise'><b>Exercise 3.6.2 [2 pts]: Data Reliability</b>
    
Having looked at the results from Exercise 3.5 (i.e., `covid_updated` DataFrame), do you think the original data is reliable and accurate? Are there any potential biases that you're aware of or concerned about? Please explain (3-5 sentences).</div>

<div style='background-color:#F6FEFA;padding:15px'>

The original data can help predict general trends. However, testing rates, false positives, access to medical care, and accuracy of population data are not represented within our data points. Data for counties with small populations could be very sensitive to a single case or death and may not accurately represent covid effects in that county.

</div>


<div class='exercise'><b>Exercise 3.6.3 [1 pt]: Relationships Between Variables</b>
    
If a county has 15 confirmed deaths, how many cases would you expect? What would you expect its population to be? Explain why (1-2 sentences in total)?

**NOTE:** For this question, we aren't evaluating the accuracy of your answer but your thought-process and reasoning.
</div>

<div style='background-color:#F6FEFA;padding:15px'>

Given the highest case fatality rate of 14%, the number of cases could be as low as 100, but with a 1/10th of the highest rate, number of cases could be up to 1000. Given the highest and lowest death rates in each state, I would expect population to be between 5k and 43.5k. 


</div>


<div class='exercise'><b>Exercise 3.6.4 [1 pt]: Further Questions</b>
    
What further questions do you wish to answer about COVID, including ones that may not be possible to answer from this data alone (e.g., Is there a correlation between the average age of people in a county and the # of COVID deaths)? Write at least 3 of your questions.</div>

<div style='background-color:#F6FEFA;padding:15px'>

Is there a correlation between the percentage of minorities in a county and death rate? Is there a correlation between presidential pick in a county and case rate? Is there a correlation between the population density in a county and case rate?

</div>


## 4. MORE DATA (25 pts)
In order to better understand how COVID (and the testing thereof) has impacted our world, we could look at how it relates to demographics, income, education, health, and political voting. For this exercise, we will make use of `election2020_by_county.csv`.

<div class='exercise'><b>Exercise 4.1 [4 pts]: Load more data</b>

Complete the `merge_data()` function, which should:
1. First, load `election2020_by_county.csv` as a new DataFrame.
2. Then, using the state and county names (case-sensitive) in both DataFrames, merge this new DataFrame with your existing `covid_updated`.
3. Return the merged DataFrame

The returned `merged` DataFrame should contain all 8 columns from `covid_updated`:
- county
- state
- \# total covid cases
- \# covid cases per 100k
- \# covid deaths
- population
- \# covid deaths per 100k
- case_fatality_rate

along with these 15 columns from `election2020_by_county.csv`:
- hispanic
- minority
- female
- unemployed
- income
- nodegree
- bachelor
- inactivity
- obesity
- density
- cancer
- voter_turnout
- voter_gap
- trump
- biden

**NOTES:**
- We are dropping two columns from `election2020_by_county.csv`:
    - fipscode
    - population
- Do not attempt to manually fix any of the state or county names. That is, **our merging should require the state and county names to be identical (case-sensitive) between the two DataFrames.** If there is a discrepancy between the two, do not worry about adjusting these names to find a perfect match.

**HINT:** there are many ways to solve this, but you may find the [pandas.merge()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) function can be really helpful

**EXTRA INFORMATION:** In case you're wondering what the different features/columns are in `election2020_by_county.csv`:

- state: the state in which the county lies
- fipscode: an ID to identify each county
- county: the name of each county
- population: total population
- hispanic: percent of adults that are hispanic
- minority: percent of adults that are nonwhite
- female: percent of adults that are female
- unemployed: unemployment rate, as a percent
- income: median income
- nodegree: percent of adults who have not completed high school
- bachelor: percent of adults with a bachelor’s degree
- inactive: percent of adults who do not exercise in their leisure time
- obesity: percent of adults with BMI > 30
- density: population density, persons per square mile of land
- cancer: prevalence of cancer per 100,000 individuals
- voter_turnout: percentage of voting age population that voted
- voter_gap: percentage point gap in 2020 presidential voting: trump-briden
</div>

In [34]:
def merge_data(covid_updated, filepath):
    
    # YOUR CODE HERE
    election_data = pd.read_csv(filepath)
    election_data = election_data.drop(columns=['fipscode','population'])
    covid_updated = covid_updated.merge(election_data, on =['state','county'])

    # END OF YOUR CODE HERE
    return covid_updated

Run the cell below (no changes necessary) to execute your code above

In [35]:
merged = merge_data(covid_updated, 'election2020_by_county.csv')

In [36]:
merged.head()

,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,state,county,hispanic,minority,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
0,0.0,0.0,3380.0,6479.289941,219,0,Nebraska,Harlan County,3.7,5.8,...,9.3,14.5,26.7,32.1,16.8,204.0,28.086535,69.2,83.8,14.6
1,0.0,0.0,465.0,4516.129032,21,0,Nebraska,Blaine County,0.0,2.0,...,1.3,15.7,25.4,29.1,0.9,NaN,19.949495,77.3,88.3,11.0
2,0.0,0.0,1392.0,5172.413793,72,0,Colorado,Jackson County,24.4,25.1,...,15.5,17.5,20.5,21.4,0.9,207.1,12.874251,58.1,77.9,19.8
3,0.0,0.0,664.0,6325.301205,42,0,Nebraska,Loup County,0.0,0.4,...,6.2,14.4,33.0,30.7,1.3,NaN,-4.849885,65.0,81.5,16.5
4,0.0,0.0,5463.0,3752.516932,205,0,Minnesota,Cook County,2.1,13.9,...,4.8,40.6,17.4,28.9,8.7,263.7,11.108502,-34.1,31.8,65.9


In [37]:
merged.shape

(3011, 23)

As mentioned above, the merging requires exact matching between the two DataFrames' `state` and `county` columns. Thus, some mismatches will occur, yielding our `merged` DataFrame to have fewer rows than `covid_updated` and `election2016_by_county.csv`.

<div class='exercise'><b>Data Construction / Understanding</b>
</div>

<div class='exercise'><b>Exercise 4.2.1 [1 pt]: Lost Rows</b>
    
Compared to `covid_updated`, how many rows were lost during this merging process to create `merged`? Running the cell below should print to the screen your answer.
</div>

In [38]:
# YOUR CODE HERE
total_loss = len(covid_updated)-len(merged)
print(str(total_loss))

# END OF YOUR CODE HERE

68


<div class='exercise'><b>Exercise 4.2.2 [2 pts]: Lost Counties</b>  

List the county and state of *at least 3* such rows that exist in `covid_updated` but didn't make it into `merged`. Running the cell below should print to the screen your answer.
</div>

In [39]:
# YOUR CODE HERE
new_covid = covid_updated['county'].isin(merged['county'])
false_new_covid = covid_updated.loc[new_covid==False]

print(false_new_covid['county']+", "+false_new_covid['state'])
# END OF YOUR CODE HERE

78                 Kusilvak Census Area, Alaska
68           Aleutians West Census Area, Alaska
1108                  Cameron Parish, Louisiana
84      Southeast Fairbanks Census Area, Alaska
1133              Plaquemines Parish, Louisiana
                         ...                   
1110                Claiborne Parish, Louisiana
1115           East Feliciana Parish, Louisiana
1117                 Franklin Parish, Louisiana
1136                Red River Parish, Louisiana
1103                Bienville Parish, Louisiana
Length: 68, dtype: object


<div class='exercise'><b>Exercise 4.2.3 [2 pts]: Suggested Fixes</b>
   
If we needed to be highly thorough and needed comprehensive data coverage, do you have any suggestions on how we could quickly, soundly fix most or all of them? (Write 2-3 sentences.)
    
<b>NOTE: Please do not actually fix these mismatches; for this Exercise, it's okay that the `merged` DataFrame is smaller than `covid_updated`</b>
</div>

<div style='background-color:#F6FEFA;padding:15px'>

Check whether each county is in election2020_by_county. If the county is found, add the coorresponding columns. Else store NaN in coorresponding columns. Or possibilty insert each column into the dataframe using a for loop. 

</div>


This past example demonstrates how easy it is for data to become messy. It also shows the importance of paying close attention to your data in order to understand what you are working with.

Our `case_fatality_rate` column can be viewed as an approximation of how effective and thorough *COVID testing* is for a given county.

Our `# covid deaths` column can be viewed as an extreme indication of how severe *COVID* has impacted a given county.

Our `# covid cases per 100k` column be viewed as middle-ground between the two aforementioned features. That is, it measures the impact of the disease and is influenced by the thoroughness of COVID testing.

Using these three informative features, we can inspect how impacted each county is, while correlating this with other features of each county, such as income-level, health metrics, demographics, etc. 

<div class='exercise'><b>Exercise 4.3 [2 pts]: Cleaning the data</b>

Before we do any further analysis, we first notice that some counties haven't encountered a single COVID death (usually ones with very small populations), thus providing us with little information. Write code in the cell below to update the `merged` DataFrame so that all rows with 0 deaths are removed.

In [40]:
# YOUR CODE HERE
merged = merged.drop(merged.index[merged['total_deaths']== 0]) # drop rows where total deaths is zero

# END OF YOUR CODE HERE

Running `.describe()` allows us to quickly see summary statistics of our DataFrame

In [41]:
merged.describe()

,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,2979.000000,2979.000000,2.979000e+03,2979.000000,2.979000e+03,2979.000000,2978.000000,2978.000000,2978.000000,2979.000000,...,2979.000000,2979.000000,2979.000000,2979.000000,2979.000000,2938.000000,2947.000000,2946.000000,2946.000000,2946.000000
mean,0.018391,226.377761,1.062667e+05,12530.934164,1.291265e+04,207.090634,9.269913,22.521760,49.920151,5.488587,...,14.979859,19.993622,25.969151,30.993689,230.500705,228.605242,35.514568,33.039477,65.658282,32.618805
std,0.009866,118.146874,3.406025e+05,3653.543575,4.404783e+04,778.061648,13.934891,19.803921,2.357593,1.955033,...,6.746965,8.729129,5.157168,4.452694,1712.483148,55.843412,13.829715,30.882108,15.504336,15.387119
min,0.000978,5.533039,4.040000e+02,2240.000000,1.400000e+01,1.000000,0.000000,0.200000,19.166215,1.800000,...,1.900000,4.400000,8.100000,11.800000,0.100000,46.200000,-168.323353,-90.000000,4.000000,3.100000
25%,0.012195,141.793708,1.121350e+04,10282.597073,1.365500e+03,23.000000,2.000000,6.900000,49.465408,4.100000,...,9.900000,14.000000,22.700000,28.400000,17.300000,193.500000,27.701651,15.450000,56.825000,20.800000
50%,0.016529,210.489388,2.627100e+04,12459.203655,3.248000e+03,56.000000,4.000000,15.200000,50.384479,5.300000,...,13.500000,17.900000,25.800000,31.200000,45.000000,230.350000,35.049804,39.300000,68.800000,29.500000
75%,0.022542,290.851113,6.874050e+04,14656.170730,8.529500e+03,135.000000,9.500000,33.700000,51.069106,6.500000,...,19.200000,23.600000,29.400000,33.800000,110.350000,265.075000,42.476078,56.875000,77.575000,41.500000
max,0.142857,865.800866,1.003911e+07,72727.272727,1.363508e+06,25579.000000,99.200000,99.400000,56.633907,24.000000,...,53.300000,72.000000,41.400000,47.600000,69468.400000,458.300000,100.000000,93.100000,96.200000,94.000000


Using the information reported from `.describe()`, we can imagine dividing our DataFrame into 4 separate bins, based on the distribution for any given feature. Specifically, based on a particular feature:
- the $1^{st}$ bin will be the data that has values between the **min** and **25%**
- the $2^{nd}$ bin will be the data that has values between **25%** and **50%**
- the $3^{rd}$ bin will be the data that has values between **50%** and **75%**
- the $4^{th}$ bin will be the data that has values between **75%** and **max**

<div class='exercise'><b>Exercise 4.4 [3 pts]: Partitioning our data</b>
    
Complete the `partition_df()` function, which takes as input:
- DataFrame to work with
- feature (e.g., obesity) to filter by
- minimum value
- maximum value

and outputs:
- a subset of the DataFrame that has values between the passed-in minimum and maximum values (inclusively) for the passed-in feature.

For example, if we called `partition_df(merged, 'obesity', 30, 45)`, it should return a subset of the `merged` DataFrame that has obesity values between 30 and 45 (and including the boundary values of 30 and 45).
</div>

In [42]:
def partition_df(df, column_name, minv, maxv):
    # YOUR CODE HERE
    subset = df.loc[(df[column_name] >= minv) & (df[column_name] <= maxv)]
    
    return subset

    # END OF YOUR CODE HERE
test = partition_df(merged, 'obesity', 30, 45)
test.describe()

,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,1840.000000,1840.000000,1.840000e+03,1840.000000,1840.000000,1840.000000,1839.000000,1839.000000,1839.000000,1840.000000,...,1840.000000,1840.000000,1840.000000,1840.000000,1840.000000,1821.000000,1824.000000,1823.000000,1823.000000,1823.000000
mean,0.018913,243.628919,5.659316e+04,13176.565939,7333.252174,124.446196,6.165851,21.223763,50.003065,5.724946,...,16.075435,16.976902,28.131848,33.629402,188.286848,236.669467,37.381176,37.734229,68.041306,30.307076
std,0.008940,111.684866,1.165225e+05,3445.857299,14808.655774,289.844254,9.616838,19.509467,2.298844,1.959587,...,6.451740,5.538305,4.398184,2.711520,1853.674512,51.541895,9.765506,26.581243,13.305524,13.285605
min,0.000978,14.060742,4.940000e+02,4108.567594,29.000000,1.000000,0.000000,0.400000,19.166215,1.800000,...,3.100000,4.400000,15.600000,30.000000,0.100000,61.800000,-27.093596,-80.800000,8.800000,5.400000
25%,0.013099,164.654560,1.154325e+04,11026.309816,1462.500000,26.000000,1.700000,6.000000,49.539643,4.400000,...,11.100000,13.075000,25.000000,31.500000,20.575000,203.900000,30.913250,23.450000,60.800000,20.500000
50%,0.017267,227.744987,2.447800e+04,13129.598804,3168.000000,57.000000,3.000000,12.900000,50.429799,5.600000,...,15.000000,16.100000,28.000000,33.100000,45.600000,237.700000,37.241643,43.100000,70.700000,27.700000
75%,0.022773,302.374252,5.159450e+04,15124.179356,6699.750000,115.000000,6.000000,32.750000,51.117133,6.800000,...,20.600000,19.900000,31.100000,35.200000,100.300000,268.300000,43.024727,57.400000,77.900000,37.400000
max,0.083612,839.541208,1.749343e+06,72727.272727,191630.000000,6654.000000,95.500000,96.500000,56.633907,21.800000,...,44.200000,43.300000,41.400000,44.900000,69468.400000,458.300000,100.000000,87.900000,93.300000,89.600000


<div class='exercise'><b>Exercise 4.5: [4 pts] Exploratory Data Analysis</b>
    
Identify a few features that you're interested in, and inspect if there's any correlation with the COVID data. Specifically, simply run your `partition_df()` function below, many times, each with a different subset of the data -- select a range of values and a particular feature. For example, if I'm interested in __cancer__, I could look at the 4 quartiles (per `.describe()`) and use those ranges of values as I repeatedly execute `partition_df()`. For this exercise, after running the function several times, **write 3-5 sentences about any patterns or correlations you noticed or didn't notice but expected to find.**
</div>

In [43]:
# YOUR CODE HERE
partition_df(merged, 'minority', 80, 100).describe()
#partition_df(merged, 'minority', 40, 60).describe()
#partition_df(merged, 'minority', 0, 20).describe()

#partition_df(merged, 'density', 110, 70000).describe()
#partition_df(merged, 'density', 0, 21).describe()

#partition_df(merged, 'biden', 41, 100).describe()
#partition_df(merged, 'biden', 0, 21).describe()
# END OF YOUR CODE HERE

#partition_df(merged, 'your feature here', your_min_value, your_max_va).describe()

,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
count,49.000000,49.000000,4.900000e+01,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,...,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,46.000000,46.000000,46.000000,46.000000
mean,0.023691,364.959266,1.776276e+05,15859.576732,30345.346939,563.306122,53.769388,87.544898,49.702275,8.857143,...,28.320408,14.446939,25.187755,31.957143,139.610204,203.412245,45.732579,-26.039130,36.273913,62.313043
std,0.009325,153.824578,4.685399e+05,4945.905709,97297.913951,1402.095682,39.338262,4.784744,3.260268,3.847185,...,9.718667,5.027761,4.414259,6.978598,415.463352,59.368856,10.197428,28.625082,14.483883,14.178177
min,0.003394,39.364913,4.040000e+02,5264.322053,56.000000,2.000000,0.100000,80.000000,39.571361,2.800000,...,8.900000,7.000000,17.700000,21.000000,0.200000,99.200000,26.581622,-80.800000,8.800000,32.100000
25%,0.017956,275.054658,7.093000e+03,12906.976744,885.000000,23.000000,4.200000,83.800000,49.284068,6.100000,...,20.500000,10.700000,22.500000,27.300000,6.700000,160.000000,39.036052,-44.175000,26.950000,53.300000
50%,0.023267,365.384615,1.597600e+04,15265.957447,2316.000000,47.000000,78.400000,87.100000,50.521485,7.900000,...,26.900000,13.400000,24.400000,29.900000,17.000000,204.900000,46.809192,-29.800000,34.300000,63.750000
75%,0.029101,468.208634,5.872200e+04,18484.666007,11020.000000,316.000000,86.500000,90.700000,51.454302,10.800000,...,35.000000,17.400000,25.800000,35.200000,45.500000,246.800000,53.630120,-7.650000,45.725000,68.300000
max,0.041653,713.557594,2.716940e+06,35746.740419,642199.000000,6654.000000,99.200000,99.400000,53.853704,24.000000,...,53.300000,30.400000,37.500000,47.600000,1978.200000,343.200000,61.487132,34.500000,66.600000,89.600000


## <div style='background-color:#F6FEFA;padding:15px'>
**Answers**    

**Minority Proportion vs Case Fatality Rate**
    
The areas that have a minority proportion of greater than 80% have a case fatality rate that is 1.4x higher (0.024) than areas that have less than 20% minorities (0.017). Though the sample sizes of the two groups are vastly different, the mean and median for both these groups are within 5% of eachother, indicating that the mean likely isn't being skewed by a single datapoint. This supports our thoughts that minorities are experiencing disproportionately higher deaths. However, direct reports of minority case fatality rate vs majority case fatality rate would allow for a more clear comparison. 
    
**High Density Areas vs Covid Cases per 100k**

The lowest density quartile has a mean case rate per 100k (12.7k)that is 1.05x higher than the highest density quartile (12.2k). The mean and median are within less than 1% of eachother, indicating that the data is not likely skewed by a few points. This was surprising to us as we expected there to be a higher case rate in more densely populated areas. 
    
**Biden Vote vs Case Rate**

The lowest Biden quartile has a mean case rate per 100k (13.3k) that is 1.15x higher than the highest Biden quartile (11.6k). The mean and median are within less than 1% of eachother, indicating that the data is not likely skewed by a few points. This is not as large of a difference as we expected due to the recent news reports of high case numbers in areas with fewer Biden supporters. 

</div>


`.describe()` provides these nice summary statistics over any portion of data that we give it. Instead of iteratively inspecting several subsets of the data, let's actually split our DataFrame into new categories; instead of representing all features by floating point numbers, let's create new _categorical_ names for feature(s) based on their numbers. The code below does just this. It creates a new column, `income group` that has 4 possible values, each one corresponding to a quartile of the original `income` values. 

Run the cell below.

In [44]:
bins = [0, 38000, 45000, 52000, 200000]
names = ['income-group-1', 'income-group-2', 'income-group-3', 'income-group-4']
d = dict(enumerate(names, 1))
merged['income group'] = np.vectorize(d.get)(np.digitize(merged['income'], bins))
merged

,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,state,county,hispanic,minority,...,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden,income group
32,0.000978,14.060742,14224.0,14370.078740,2044,2,Nebraska,Saline County,24.3,29.3,...,14.3,31.0,33.0,1.5,309.1,40.293874,28.5,62.9,34.4,income-group-3
33,0.001073,12.304663,8127.0,11467.946352,932,1,Colorado,Lake County,33.9,35.7,...,30.3,15.1,17.5,19.4,112.4,29.853937,-20.2,37.9,58.1,income-group-3
34,0.001283,14.330754,20934.0,11168.434126,2338,3,Minnesota,Dodge County,4.9,7.5,...,24.1,18.3,24.9,709.0,140.3,17.427640,30.5,64.0,33.5,income-group-4
35,0.001385,22.513649,17767.0,16254.854506,2888,4,Colorado,Pitkin County,9.8,14.3,...,56.4,8.9,14.9,17.7,70.8,12.694664,-52.1,23.2,75.3,income-group-4
36,0.001437,14.066676,7109.0,9790.406527,696,1,West Virginia,Calhoun County,1.3,2.7,...,8.9,30.2,32.7,27.3,304.8,50.167729,60.5,79.6,19.1,income-group-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,0.080313,548.410580,20605.0,6828.439699,1407,113,Georgia,Dodge County,3.4,34.7,...,14.4,28.8,28.5,44.0,212.3,51.126453,45.5,72.4,26.9,income-group-1
3007,0.083612,579.038796,8635.0,6925.303995,598,50,Georgia,Wilcox County,4.3,41.5,...,8.7,27.7,31.5,24.5,248.4,53.295374,46.9,73.2,26.3,income-group-1
3008,0.097561,642.054575,623.0,6581.059390,41,4,Nebraska,Grant County,1.9,3.8,...,18.6,30.6,28.3,4.5,305.1,23.574144,88.3,93.3,5.0,income-group-3
3009,0.110465,639.515315,2971.0,5789.296533,172,19,Georgia,Glascock County,1.6,12.6,...,11.4,24.8,28.6,21.4,264.6,32.529082,79.7,89.6,9.9,income-group-2


<div class='exercise'><b>Exercise 4.6 [5 pts]: Aggregate data</b>
    
    
Write code in the cell below to group (and display) the data according to the 4 income groups. Also, while we will still keep the same columns (i.e, features), the values of each should now represent the __average__ value of all rows that were subsumed in the making of the aggregate income-group. Your resulting DataFrame should have just 4 rows (income-group-1, income-group-2, income-group-3, income-group-4). See example in the cell below.


Since every feature (except for `# total cases`, `# covid deaths`, and `population`) was already an average value corresponding to a particular __county__, when we aggregate our data by income groups, we are effectively taking an average of an average. Many counties are being aggregated for each income-group row. This approach isn't as accurate as possible; it would be more accurate if we re-adjusted every value so that it was truly an average that was based on the total __population__ of all counties that are subsumed within a given income-group row. That's okay, though. An average of averages will suffice for the purpose of this exercise. 
</div>

In [45]:
# EXAMPLE: If our `merged` DataFrame were
# COUNTY    INCOME GROUP    BACHELOR ... (other columns, too)
#   A            2             50
#   B            1             20
#   C            1             30
#   D            2             70
#   E            3             95

# it should become
# INCOME GROUP    BACHELOR ... (other columns, too)
#   1                25
#   2                60
#   3                95

# YOUR CODE HERE
merged.groupby(by = ["income group"], axis = 0).mean() # sum the columns per each state
# END OF YOUR CODE HERE

,case_fatality_rate,total_deaths_per_100k,population,total_cases_per_100k,total_cases,total_deaths,hispanic,minority,female,unemployed,...,nodegree,bachelor,inactivity,obesity,density,cancer,voter_turnout,voter_gap,trump,biden
income group,,,,,,,,,,,,,,,,,,,,,
income-group-1,0.022794,294.207537,27822.511450,13696.377124,3903.624427,84.635115,9.138779,32.366718,50.002613,7.272366,...,21.991145,13.595420,30.101985,33.647328,87.845954,238.566258,41.844069,33.203988,65.926074,32.722086
income-group-2,0.019133,235.464740,64545.897909,12578.764281,8565.934809,143.458795,8.869704,20.386946,49.813455,5.718696,...,15.617712,17.565068,26.996187,31.489914,131.076630,235.296981,37.075902,40.729543,69.545117,28.815575
income-group-3,0.017057,208.969399,98807.927476,12364.863794,12812.709902,192.055788,8.860669,18.001534,49.863219,4.852301,...,12.760530,20.438773,25.085914,30.619386,341.441562,227.691926,34.282463,35.691831,66.912254,31.220423
income-group-4,0.015205,176.838188,220432.837531,11670.507753,24885.494962,386.840050,10.156927,20.665365,50.012650,4.356045,...,10.546977,27.356423,22.305793,28.634509,349.803023,214.376306,29.680387,22.443742,60.226839,37.783097


<div class='exercise'><b>Wrapping Up</b>
</div>

<div class='exercise'><b>Exercise 4.7.1 [1 pt]: Conclusions</b>
What are your conclusions/finding from this alternative view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**

The counties in the lowest income quartile have 1.2x more covid cases per 100k on average and one is 1.5x more likely to die from covid on average. Those counties also are more affected by possible covid risk factors such as cancer, obesity and inactivity. While this may play a role, it is unlikely that these are the sole factors. They also have lower population density, which could indicate their populations may be farther from key resources like medical care. Additionally, medical care is likely less financially acceptable to them. A multilinear model and the associated feature importance graph may be a godd way to gain a better understanding of what has the largest influence on death and fatality rates.  

</div>


<div class='exercise'><b>Exercise 4.7.2 [1 pt]: Possible Weaknesses</b>
What are some weaknesses from this view of the data? (2-4 sentences).
</div>

<div style='background-color:#F6FEFA;padding:15px'>

**your answer here**
    
When only looking at mean values, it is hard to get a sense of whether the data is being skewed by an outlier. For example, the lowest income quartile case and death rates could be skewed by a single severe outbreak of Covid in a single county that limited treatment resources. 

</div>


## Moving Forward

In this homework assignment, we've focused on gathering, parsing, and exploring data. However, what if we wanted to *predict* some behavior of the data. For example, imagine one is curious how a particular county will respond to COVID. Or, imagine we looked at counties' COVID data on a weekly basis, one could be interested in predicting the upcoming week's behavior.

Alternatively, one could be interested in *inference*, whereby we are more concerned with trying to understand __why__ and __how__ a system behaves the way it does. We might wish to understand which factors most correlate and cause a certain event to happen. This could give us insights into where certain inequalities persist.

For both *prediction* and *inference*, our computational method of solving such a task is referred to as a model. For the remainder of CS109, we will spend significant focus on various models.
</div>

## Reflection

As a reminder, this is just **one** of the homework assignments in this course, the point of which is to assess your learning and to provide both you and us with an indication as to how aligned your knowledge and skills are with our learning objectives. To this end, we encourage you to reflect on your progress, strengths, and weaknesses and to make changes, if necessary, to accomplish your goals. Likewise, please reach out to the TFs and teaching staff if you need help. We want everyone to feel comfortable in being honest about these elements, with both herself/himself and us. For these purposes, we will ask you several times throughout the semester to complete an anonymous poll.